In [ ]:
import pickle
import hashlib
import bisect
import os
import json

class PickleShardManager:
    def __init__(self, num_shards=3, folder="data_shards"):
        self.num_shards = num_shards
        self.folder = folder
        self.metadata_file = f"{folder}/metadata.json"
        self.shard_keys = []
        self.shards = {}
        self.metadata = {}

        os.makedirs(folder, exist_ok=True)

        # Khởi tạo shards và sắp xếp để dùng bisect
        for i in range(num_shards):
            shard_key = self._hash(f"shard-{i}")
            self.shard_keys.append(shard_key)
            self.shards[shard_key] = f"{folder}/shard_{i}.pkl"

        self.shard_keys.sort()
        self._load_metadata()

    def _hash(self, key):
        """Tạo hash integer từ key"""
        return int(hashlib.md5(key.encode()).hexdigest(), 16)

    def _get_shard_file(self, key):
        """Tìm file pickle phù hợp"""
        key_hash = self._hash(key)
        idx = bisect.bisect(self.shard_keys, key_hash) % self.num_shards
        return self.shards[self.shard_keys[idx]]

    def _save_metadata(self):
        """Lưu metadata dạng JSON thay vì pickle"""
        with open(self.metadata_file, "w") as f:
            json.dump(self.metadata, f)

    def _load_metadata(self):
        """Tải metadata nếu tồn tại"""
        if os.path.exists(self.metadata_file):
            with open(self.metadata_file, "r") as f:
                self.metadata = json.load(f)

    def save(self, key, data):
        """Lưu dữ liệu vào file pickle phù hợp và cập nhật metadata"""
        if key in self.metadata:
            raise KeyError(f"Key '{key}' đã tồn tại! Dùng update() thay vì save().")

        file_path = self._get_shard_file(key)
        self.metadata[key] = file_path
        self._save_metadata()

        # Lưu vào file pickle theo block (tránh duplicate nếu file lớn)
        all_data = self.load_shard(file_path)
        all_data[key] = data
        with open(file_path, "wb") as f:
            pickle.dump(all_data, f)

    def exists(self, key):
        """Kiểm tra nhanh key có tồn tại không"""
        return key in self.metadata

    def find_by_key(self, key):
        """Lấy dữ liệu theo key"""
        if not self.exists(key):
            raise KeyError(f"Key '{key}' không tồn tại!")

        file_path = self.metadata[key]
        data = self.load_shard(file_path)
        return data.get(key, None)

    def update(self, key, new_data):
        """Cập nhật dữ liệu của một key"""
        if not self.exists(key):
            raise KeyError(f"Key '{key}' không tồn tại!")

        file_path = self.metadata[key]

        # Đọc dữ liệu hiện tại, cập nhật
        data = self.load_shard(file_path)
        if key in data:
            data[key].update(new_data)
        else:
            data[key] = new_data

        # Ghi đè lại file với dữ liệu mới
        with open(file_path, "wb") as f:
            pickle.dump(data, f)

    def load_shard(self, file_path):
        """Đọc toàn bộ dữ liệu từ một shard"""
        if not os.path.exists(file_path):
            return {}

        with open(file_path, "rb") as f:
            try:
                return pickle.load(f)
            except EOFError:
                return {}

    def load_all(self):
        """Đọc toàn bộ dữ liệu từ tất cả các shards"""
        all_data = {}
        for file_path in self.shards.values():
            all_data.update(self.load_shard(file_path))
        return all_data


In [2]:
manager = PickleShardManager(num_shards=3)

In [3]:
manager.save("1", [1,2,3])